## Creating Audio File

Checkout https://beta.elevenlabs.io/history for full log of audio creation

In [3]:
file_name = "./kitten_full"
mp3_file_name = file_name + ".mp3"
wav_file_name = file_name + ".wav"

generate_new_audio = False
name_of_voice = "Bella"
text = "I love you"

In [23]:
# Get voice ID
import requests

r1 = requests.get('https://api.elevenlabs.io/v1/voices')
resp = r1.json()
rid = ""
for voice in resp["voices"]:
    if voice["name"] == name_of_voice:
        print(voice)
        rid = voice["voice_id"]
        break;
rid

{'voice_id': 'EXAVITQu4vr4xnSDxMaL', 'name': 'Bella', 'samples': None, 'category': 'premade', 'fine_tuning': {'is_allowed_to_fine_tune': False, 'fine_tuning_requested': False, 'finetuning_state': 'not_started', 'verification_attempts_count': 0}, 'labels': {}, 'preview_url': 'https://storage.googleapis.com/eleven-public-prod/premade/voices/EXAVITQu4vr4xnSDxMaL/04365bce-98cc-4e99-9f10-56b60680cda9.mp3', 'available_for_tiers': [], 'settings': None}


'EXAVITQu4vr4xnSDxMaL'

In [5]:
# Get audio file, minimize runs cost money
import json

if generate_new_audio:
    url = "https://api.elevenlabs.io/v1/text-to-speech/{voice_id}".format(voice_id=rid)
    print(url)
    headers = {'xi-api-key': "aa42307ce6bfbefcfd2abd4d8634127b"}
    payload = {
      "text": text,
      "voice_settings": {
        "stability": 0.3,
        "similarity_boost": 0.75
      }
    }

    r2 = requests.post(url, data=json.dumps(payload), headers=headers)
    print(r2)

In [6]:
# Save and play audio file
import os

if generate_new_audio:
    with open(mp3_file_name, 'wb') as f:
        f.write(r2.content)

    os.system("mpg123 " + mp3_file_name)

In [7]:
# convert mp3 to wav                                                            
from pydub import AudioSegment
from pydub.playback import play

if generate_new_audio:
    sound = AudioSegment.from_mp3(mp3_file_name)
    sound.export(wav_file_name, format="wav")

    # test sound
    song = AudioSegment.from_wav(wav_file_name)
    play(song)

## Generate phonemes
- Uses https://github.com/m-bain/whisperX

In [9]:
# no extension, must be wav
import os
cwd = os.getcwd() #pwd
print(cwd)

audio_file_name = wav_file_name
base_audio_file_name = os.path.basename(audio_file_name)
print("File name:", audio_file_name)
print("Base name:", base_audio_file_name)

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/scripts
File name: ./kitten_full.wav
Base name: kitten_full.wav


In [10]:
import whisperx

device = "cpu" 
# transcribe with original whisper
model = whisperx.load_model("large", device)

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/torchaudio/_internal/module_utils.py:99: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [11]:
result = model.transcribe(audio_file_name)

print(result["segments"]) # before alignment

# load alignment model and metadata
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)

# align whisper output
result_aligned = whisperx.align(result["segments"], model_a, metadata, audio_file_name, device)

# print(result_aligned["segments"]) # after alignment
result_aligned["word_segments"]

/Users/jfuentes/Projects/infinitenews/AAAI22-one-shot-talking-face/venv/lib/python3.9/site-packages/whisperx/transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 1.4000000000000001, 'text': ' Good evening, everyone.', 'tokens': [2205, 5634, 11, 1518, 13], 'temperature': 0.0, 'avg_logprob': -0.29594694829620094, 'compression_ratio': 1.6071428571428572, 'no_speech_prob': 0.0020815161988139153}, {'id': 1, 'seek': 0, 'start': 1.4000000000000001, 'end': 5.96, 'text': ' Sidney Scott here bringing you the latest news from the upside down world.', 'tokens': [19797, 2397, 6659, 510, 5062, 291, 264, 6792, 2583, 490, 264, 14119, 760, 1002, 13], 'temperature': 0.0, 'avg_logprob': -0.29594694829620094, 'compression_ratio': 1.6071428571428572, 'no_speech_prob': 0.0020815161988139153}, {'id': 2, 'seek': 0, 'start': 5.96, 'end': 11.0, 'text': ' It appears that Ponzi schemes, once thought to be a financial fraud tool used', 'tokens': [467, 7038, 300, 31756, 3992, 26954, 11, 1564, 1194, 281, 312, 257, 4669, 14560, 2290, 1143], 'temperature': 0.0, 'avg_logprob': -0.29594694829620094, 'compression_ratio': 1.60714285714285

[{'text': 'Good', 'start': 0.10144927536231885, 'end': 0.24347826086956526},
 {'text': 'evening,', 'start': 0.30434782608695654, 'end': 0.5478260869565218},
 {'text': 'everyone.', 'start': 0.6086956521739131, 'end': 0.9942028985507247},
 {'text': 'Sidney', 'start': 1.5205286343612336, 'end': 1.8218502202643172},
 {'text': 'Scott', 'start': 1.8620264317180617, 'end': 2.20352422907489},
 {'text': 'here', 'start': 2.2437004405286345, 'end': 2.4244933920704845},
 {'text': 'bringing', 'start': 2.4847577092511015, 'end': 2.786079295154185},
 {'text': 'you', 'start': 2.8262555066079296, 'end': 2.946784140969163},
 {'text': 'the', 'start': 3.0070484581497796, 'end': 3.0874008810572686},
 {'text': 'latest', 'start': 3.127577092511013, 'end': 3.4489867841409687},
 {'text': 'news', 'start': 3.5293392070484577, 'end': 3.7503083700440527},
 {'text': 'from', 'start': 3.8105726872246697, 'end': 3.9511894273127757},
 {'text': 'the', 'start': 3.9712775330396477, 'end': 4.071718061674009},
 {'text': 'up

In [106]:
test_words = ["that's", "Help! ", "multiverse", "cat", "permit", "surprise", ",whales"]

In [107]:
import nltk

try:
    arpabet = nltk.corpus.cmudict.dict()
except LookupError:
    nltk.download('cmudict')
    arpabet = nltk.corpus.cmudict.dict()

In [108]:
import re
from functools import lru_cache
from itertools import product as iterprod

def cleanWord(word):
    regex = r"[^\w\']"
    return re.sub(regex, "", word).lower()

def arpabetWithoutNumbers(word):
    list_of_phones = []
    for phones in arpabet[word]:
        new_phones = ["".join(filter(lambda x: x.isalpha(), phone)) for phone in phones]
        list_of_phones.append(new_phones)
    return list_of_phones

def wordbreak(s):
    s = cleanWord(s)
    if s in arpabet:
        return arpabetWithoutNumbers(s)
    middle = len(s)/2
    partition = sorted(list(range(len(s))), key=lambda x: (x-middle)**2-x)
    for i in partition:
        pre, suf = (s[:i], s[i:])
        if pre in arpabet and wordbreak(suf) is not None:
            return [x+y for x,y in iterprod(arpabetWithoutNumbers(pre), wordbreak(suf))]
    return None

# Example words: 
for w in test_words:
    print(wordbreak(w))

[['DH', 'AE', 'T', 'S']]
[['HH', 'EH', 'L', 'P']]
[['M', 'AH', 'L', 'T', 'IY', 'V', 'ER', 'S']]
[['K', 'AE', 'T']]
[['P', 'ER', 'M', 'IH', 'T'], ['P', 'ER', 'M', 'IH', 'T']]
[['S', 'ER', 'P', 'R', 'AY', 'Z'], ['S', 'AH', 'P', 'R', 'AY', 'Z']]
[['W', 'EY', 'L', 'Z'], ['HH', 'W', 'EY', 'L', 'Z']]


In [132]:
def letterToPhone(word, phones):
    # print("LTP", word, phones)
    if word == "":
        return []

    for i in reversed(range(1, len(word))):
        for j in range(0, len(word) - i + 1):
            subword = word[j:i+j]
            cur_possible_phones = wordbreak(subword)
            #For when has contractions
            if not cur_possible_phones:
                print("Skipping", i, j, subword, cur_possible_phones)
                continue
            else:
                cur_possible_phones = cur_possible_phones + [[subword.upper()]]
                print(i, j, subword, cur_possible_phones)

            for cur_phones in cur_possible_phones:
                extra_length = len(phones) - len(cur_phones)
#                 print("EL", extra_length)
                if extra_length > 0:
                    for a in range(0, extra_length + 1):
                        subphones_end = len(phones) - (extra_length - a)
                        subphones = phones[a:subphones_end]
                        if len(subphones) != len(cur_phones):
                            print("Lengths not equal....")
                        #print(subphones, cur_phones)
                        if subphones == cur_phones:
                            #print("MATCH", subword, subphones)
                            first = letterToPhone(word[0:j], phones[0:a])
                            cur = letterToPhone(subword, subphones)
                            last = letterToPhone(word[i+j:], phones[subphones_end:])
                            # print("LTP Complete", first, cur, last)
                            return first + cur + last
                    
    return [[word, phones]]

for segment in result_aligned["word_segments"]:
    word = segment["text"]

# for word in test_words:
    print(word)
    print(letterToPhone(word, wordbreak(word)[0]))
    print([word, wordbreak(word)[0]], "\n")

Good
3 0 Goo [['G', 'UW'], ['GOO']]
3 1 ood [['OW', 'OW', 'D', 'IY'], ['OOD']]
2 0 Go [['G', 'OW'], ['GO']]
2 1 oo [['OW', 'OW'], ['OO']]
2 2 od [['OW', 'D', 'IY'], ['OD']]
1 0 G [['JH', 'IY'], ['G']]
2 0 oo [['OW', 'OW'], ['OO']]
2 1 od [['OW', 'D', 'IY'], ['OD']]
1 0 o [['OW'], ['O']]
1 1 o [['OW'], ['O']]
1 2 d [['D', 'IY'], ['D']]
1 0 o [['OW'], ['O']]
1 1 o [['OW'], ['O']]
[['G', ['G']], ['oo', ['UH']], ['d', ['D']]]
['Good', ['G', 'UH', 'D']] 

evening,
7 0 evening [['IY', 'V', 'N', 'IH', 'NG'], ['EVENING']]
7 1 vening, [['V', 'IY', 'N', 'IH', 'NG'], ['V', 'IY', 'IY', 'N', 'IH', 'NG'], ['VENING,']]
6 0 evenin [['IY', 'V', 'N', 'IY', 'EH', 'N'], ['EVENIN']]
6 1 vening [['V', 'IY', 'N', 'IH', 'NG'], ['V', 'IY', 'IY', 'N', 'IH', 'NG'], ['VENING']]
6 2 ening, [['EH', 'N', 'IH', 'NG'], ['ENING,']]
5 0 eveni [['IY', 'V', 'N', 'IY'], ['EVENI']]
5 1 venin [['V', 'IY', 'N', 'IY', 'EH', 'N'], ['V', 'IY', 'IY', 'N', 'IY', 'EH', 'N'], ['VENIN']]
5 2 ening [['EH', 'N', 'IH', 'NG'], ['ENING']]

In [54]:
# {'text': 'Good', 'start': 0.10120481927710842, 'end': 0.32385542168674697},
# INTO
# [
#     {
#         "word": "sil",
#         "phones": [
#             {
#                 "ph": "SIL",
#                 "bg": 0,
#                 "ed": 126
#             }
#         ]
#     },
#     {
#         "word": "YOU",
#         "phones": [
#             {
#                 "ph": "Y",
#                 "bg": 126,
#                 "ed": 138
#             },
#             {
#                 "ph": "UW",
#                 "bg": 138,
#                 "ed": 141
#             }
#         ]
#     },
from math import trunc

def makeSegment(word, phones, start, end):
    total = end - start
    time_per_phone = trunc(total / len(phones))
#     print(total, time_per_phone)
    expanded_phones = [{"ph": phone, "bg": start + (i * time_per_phone), "ed": start + ((i + 1) * time_per_phone)} for i, phone in enumerate(phones)]
    return {"word": word, "phones": expanded_phones}

def makeSil(start, end):
    return {"word": "sil", "phones": [{"ph": "SIL", "bg": start, "ed": end}]}
  
all_phonemes = []
for segment in result_aligned["word_segments"]:
    word = segment["text"]
    #Get rid of numbers in phones
    phonemes = wordbreak(word)[0]
    print(phonemes)
    start_time = trunc(segment["start"] * 100)
    end_time = trunc(segment["end"] * 100)
    
    if len(all_phonemes) > 0:
        last = all_phonemes[-1]
        last_end = last["phones"][-1]["ed"]
        if start_time - last_end > 5:
            sil = makeSil(last_end, start_time)
            all_phonemes.append(sil)
        else:
            start_time = last_end
    else:
        sil = makeSil(0, start_time)
        all_phonemes.append(sil)
        
    segment = makeSegment(word, phonemes, start_time, end_time)
    all_phonemes.append(segment)

all_phonemes

['AY']
['L', 'AH', 'V']
['Y', 'UW']


[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'I', 'phones': [{'ph': 'AY', 'bg': 10, 'ed': 16}]},
 {'word': 'love',
  'phones': [{'ph': 'L', 'bg': 16, 'ed': 23},
   {'ph': 'AH', 'bg': 23, 'ed': 30},
   {'ph': 'V', 'bg': 30, 'ed': 37}]},
 {'word': 'you.',
  'phones': [{'ph': 'Y', 'bg': 37, 'ed': 43},
   {'ph': 'UW', 'bg': 43, 'ed': 49}]}]

In [139]:
import json

file_name = "../samples/phonemes/{name}.json".format(name=audio_file_name)
print("Writing to ", file_name)
f1 = open(file_name, 'w')
json.dump(all_phonemes, f1)

Writing to  ../samples/phonemes/kitten_full.json


In [55]:
all_phonemes

[{'word': 'sil', 'phones': [{'ph': 'SIL', 'bg': 0, 'ed': 10}]},
 {'word': 'I', 'phones': [{'ph': 'AY', 'bg': 10, 'ed': 16}]},
 {'word': 'love',
  'phones': [{'ph': 'L', 'bg': 16, 'ed': 23},
   {'ph': 'AH', 'bg': 23, 'ed': 30},
   {'ph': 'V', 'bg': 30, 'ed': 37}]},
 {'word': 'you.',
  'phones': [{'ph': 'Y', 'bg': 37, 'ed': 43},
   {'ph': 'UW', 'bg': 43, 'ed': 49}]}]

In [145]:
import requests

url = "http://localhost:1337/api/modules/1?populate=Image1"
print(url)
token = "5501db97b19e4096535c29375e22f346d010ac3ec69a19d7cfa944f02124b5670d05deb3a8490f8e14a0daaef654ba0e785018e4e1e6edfd22966ff99719658a9f5dfbcd8cec2b6c2fd110103c93c70fe7849e0eaf35642946c131b5f6b8cf844118d2fd1a1fded70372375f7525fb96fd664fbf0bad11a036b96bfb87b190a5"
headers = {'Authorization': 'Bearer ' + token}

r2 = requests.get(url, headers=headers)
r2.json()

http://localhost:1337/api/modules/1?populate=Image1


{'data': {'id': 1,
  'attributes': {'Text1': "Good evening everyone and welcome to this special report. As you know, we’ve been tracking a strange uptick in activity in the multiverse - and as we feared, it appears that space whales have been invading our universe through a rift in the multiverse. \n\nThese massive creatures, which can range in size from the size of a small ship to larger than a mountain, are wreaking havoc on the balance of the multiverse. What's worse is that they seem to be coming from a totally different dimension, one that our scientists didn't even know existed until now.\n\nWe’ve seen some strange things in the multiverse before, but never anything as awe-inspiring and destructive as these space whales. They’ve already destroyed several planets and star systems, and it looks like they’re only getting started. \n\nThe multiverse is in a state of crisis, but thankfully our scientists are working around the clock to try and close the rift and prevent any more of th